<a href="https://colab.research.google.com/github/lizarci3/gtp2_film_generation/blob/master/film_script_generation_gtp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

Original article can be found [here](https://towardsdatascience.com/film-script-generation-with-gpt-2-58601b00d371)

Repo [here](https://github.com/cdpierse/script_buddy_v2)

The author used film scripts (~60 MB) of data scraped from the Internet Movie Script Database (IMSDB) in order to fine-tune a GTP2 to write a film script.




The author of this script only had ~1300 scripts to use, however, on averagea screenplay has 30,000 words. So the dataset has close to 40 million sequences of words.

The author wanted the model to be able to generate entire sequences of scripts with mixed scripted elements in each sequence.

This fine-tuning is developed based on hugginface's example on fine-tuning dataset found in run_language_modeling.py

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=7ca647262279ee7387b78f6a2680e902577af13fd5d4faf20cfbc545260930eb
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  |     Proc size: 111.2 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total     15079MB


# Preparing the data

The script data is loaded into the model in batches were the data has already been tokenized for GPT-2. In the repo, the ScriptData class splits the entire dataset into tokenized blocks of tensors.

Once the data has been properly prepared, these blocks are loaded in batches into a GPT-2 in a training loop.

In [2]:
!pip install git+https://github.com/huggingface/transformers #just doing a pip install transformers creates some sync problems

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-kdinxqjq
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-kdinxqjq
     |████████████████████████████████| 3.0MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 40.7MB/s 
     |████████████████████████████████| 890kB 56.7MB/s 
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=824179 sha256=86a7ebc31e9d7ecedab959aa6bf391beb0791922552583d5b6aa0b66a14cf356
  Stored in directory: /tmp/pip-ephem-wheel-cache-7lt6fekb/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b0593864f597789038eaa5fc899fa3378652c914ec676d2e9015c6b4c137814f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import numpy as np
import os
import logging 
import pickle
import logging
import random

The gpt2-medium model used in this work has 12 layers ~345 million parameters and took ~6h to train (with 3 epochs).

The first thing that needs to be done is to upload the model and tokenizer from the pre-trained transformers package.

In [4]:
device = 'cpu'

if torch.cuda.is_available():
  device = 'cuda'

device

'cuda'

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

* In order to fine tune this pre-trained model you need to create a training loop where you progressively load a batch of script sequences from the entire dataset.
* Each batch is like a tokenized block of tensors from the data (done in ScriptData).
* An important parameter to consider is the batch size. Large batch sizes can result in running out of GPU memory fast. To start, you can choose a batch of 1 and then test how much you can test it.

* In this work his batch size was 7.

In [6]:
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from transformers import (
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)


# Preparing the Data

The following ScriptData class splits the dataset into tokenized blocks of tensors. These blocks will then be loaded in batches into the training loop.

In [7]:
FILE_PATH = "/content/drive/My Drive/WJ/film_text.txt" # ~ 60 MB
logger = logging.getLogger(__name__)

class ScriptData(Dataset):

  def __init__(
      self, #instance of the class ScriptData
      tokenizer: PreTrainedTokenizer,
      file_path: str, 
      block_size = 512, # Fine-tuning item
      overwrite_cache = False
  ):

      assert os.path.isfile(file_path) #assert raises an error if condition False

      block_size = block_size - (tokenizer.max_len - tokenizer.max_len_single_sentence)

      directory, filename = os.path.split(file_path)

      #Create the path/filename for the cached file
      # so that is stored in the same folder and it stores which block size we used
    
      cached_features_file = os.path.join(directory,"gpt2"+"_"+str(block_size)+"_"+filename)

      #if the file already exists and if overwrite_cache is set to False don't overwrite
      if os.path.exists(cached_features_file) and not overwrite_cache:
        logger.info(f"Loading features from cached file {cached_features_file}")

        with open(cached_features_file, 'rb') as cache:
          self.examples = pickle.load(cache)
          logger.debug("Loaded examples from cache")

      else:

        logger.info(f"Creating features from file {filename} at {directory}")

        self.examples = []

        with open(file_path, encoding="utf-8") as f:
          text = f.read()
          logger.debug("Succesfully read text from file")

        #convert_tokens_to_ids = Converts a token string (or a sequence of tokens) in a single integer id (or sequence of ids), using the vocabulary
        tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

        #slice in steps of block_size the text
        #append in examples

        for i in range(0, len(tokenized_text) - block_size + 1, block_size):
          self.examples.append(
              tokenizer.build_inputs_with_special_tokens( #From Bert model: 
                  tokenized_text[i : i + block_size]
              )
          )

        logger.info(f"Saving features into cached file {cached_features_file}")

        # save it
        with open(cached_features_file, "wb") as cache:
          pickle.dump(self.examples, cache, protocol=pickle.HIGHEST_PROTOCOL)

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item], dtype=torch.long)




# Fine-tuning GPT-2: Training

A GPU is necessary when training this model. We are using a dataset of film scripts that is about 60 MB to train. This text has been prepared by scrapping IMSDB (see in the repo the specifics of the scrapping).

We talked about how to fine tune the model (or optimize it on a custom dataset of tokenized text) you need to create a TRAINING LOOP WHERE YOU PROGRESSIVELY LOAD A BATCH OF SCRIPT SEQUENCES FROM THE DATASET.

* Each batch (a batch of tokenized tensor) is run through the language model head as BOTH its intput and target labels.
* From this step we return the loss and logits (i.e., prediction scores) to conduct the backward pass on the gradients.
* Every X number of batches set up an evaluation step to generate a batch of text. This helps us understand how well the model is optimizing and being fine-tuned to the specific text.
* Transformer's generate function provides a number of different decoding methods to get the best results.

In [9]:
output_dir = '/content/drive/My Drive/WJ/'

In [10]:
dataset = ScriptData(tokenizer= tokenizer, file_path= FILE_PATH )
script_loader = DataLoader(dataset,batch_size=4,shuffle=True)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1294: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


In [11]:
type(script_loader)

torch.utils.data.dataloader.DataLoader

### Parameters

In [12]:
BATCH_SIZE = 1 #starting point, author used in the end a batch_size of 7
EPOCHS = 1 # the author mentions he used in total 3 full epochs lasting ~6h for training
LEARNING_RATE = 0.00002
WARMUP_STEPS = 10000

Start the optimizer, scheduler and set up the loss, batch counts to start at zero

In [13]:
model.train()

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)



## Just for testing

In [14]:
script_count = 0
sum_loss = 0.0
batch_count = 0

for epoch in range(EPOCHS):
  print(f"EPOCH {epoch} started"+'='*30)

  j = 0
  nsamples = 199

  for idx, script in enumerate(script_loader):

    if j>nsamples:
      break

    else:
      print(idx, script, script[0].shape) #512 was what we used as block size in ScriptData

      outputs = model(script.to(device), labels=script.to(device))

      loss, logits = outputs[:2] # language modeling loss and prediction scores of the language modeling head 
      loss.backward()

      sum_loss = sum_loss + loss.detach().data
      script_count = script_count + 1
      print('Sum loss', sum_loss, 'script_count', script_count)
      
      #once we have loaded enough scripts == batch_size

      j = j + 1 

      if script_count == BATCH_SIZE:
        print('script_count equal to batch_size')
        script_count = 0 # re-start script counter
        batch_count = batch_count + 1 # how many full batches we have
        print('batch_count', batch_count)
        
        optimizer.step() # perform an optimization step
        scheduler.step()
        optimizer.zero_grad() # clear the gradients from the last step, PyTorch accumulates the gradients so before starting to propagate we need to set them to zero
        model.zero_grad()


        ## As opposed to the jokes generation script, this script generates text
        ## Apparently, every X number of batches to see how it is doing
        ## here, UNDERSTAND BELOW PARAMS
        if batch_count == 200:
            model.eval()  
            print(f"sum loss {sum_loss}")

            ## see https://huggingface.co/blog/how-to-generate

            sample_outputs = model.generate( #function added since version 2.4
                                    bos_token_id=random.randint(1,30000), # I AM ASSUMIN GIS THE INPUT: The sequence used as a prompt for the generation. If `None` the method initializes  it as an empty `torch.LongTensor` of shape `(1,)`. 
                                    do_sample=True, #If set to `False` greedy decoding is used. Otherwise sampling is used. Defaults to `False` as defined in `configuration_utils.PretrainedConfig`.  
                                    top_k=50, #The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling. Must be between 0 and 1. Default to 1.  
                                    max_length = 1000, # The max length of the sequence to be generated.  Between `min_length` and infinity. Default to 20.
                                    top_p=0.95, #The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling. Must be between 0 and 1. Default to 1.  
                                    num_return_sequences=1 #The number of independently computed returned sequences for each element in the batch. Default to 1.
                                )

            print("Output:\n" + 100 * '-')
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            batch_count = 0
            sum_loss = 0.0
            model.train()
        

        




EPOCH 0 started==============================
0 tensor([[  607,  1735,    13,  ...,  6916,  1911,   383],
        [  345,   588,   284,  ...,    13,   198,   220],
        [  220,   357,  4663,  ...,   220,   220,   220],
        [  220,   220, 26746,  ...,   262,  6546,   220]]) torch.Size([512])
Sum loss tensor(2.0897, device='cuda:0') script_count 1
script_count equal to batch_size
batch_count 1
1 tensor([[  220,   220,   220,  ...,   220,   220,   220],
        [  220,   220,   220,  ...,   344,   611,   428],
        [  220,   220,   220,  ...,   220,   220,   220],
        [  198,   220,   220,  ...,   339, 15455,   329]]) torch.Size([512])
Sum loss tensor(3.9353, device='cuda:0') script_count 1
script_count equal to batch_size
batch_count 2
2 tensor([[  628,   198,   220,  ...,   220,   220,   220],
        [  220,   220,   220,  ...,   290, 42966, 14412],
        [   11,  6451,   379,  ...,     0,   198,   220],
        [  220,   220,   220,  ...,   220,   220,   220]]) torch.S

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Sum loss tensor(441.8589, device='cuda:0') script_count 1
script_count equal to batch_size
batch_count 200
sum loss 441.85888671875
Output:
----------------------------------------------------------------------------------------------------
0:  plane on its way back from the Antarctic expedition with its commander, a man named Robert C. Hockney.

After he was arrested on Jan. 7 at the New York Federal Prison, he was transferred to a secure part of the facility at Camp Lejeune. Hockney refused a security check and called on a group of crew that consisted of a handful of members of his crew. Hockney was an Australian citizen who came from a family of Italian and French immigrants known locally as the 'Ossans.' He had just left his home and spent the night watching TV in an asylum-seeker camp. It appears that he didn't see the people of the area but rather those who had been stranded at the other camp, a place where they were supposed to live. This is apparently what happened.

A few days

# Breaking down preparing the data

In [ ]:
FILE_PATH = "/content/drive/My Drive/WJ/film_text.txt" # ~ 60 MB
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

model.train()

In [ ]:
assert os.path.isfile(FILE_PATH)

In [ ]:
tokenizer.max_len, tokenizer.max_len_single_sentence

(1024, 1024)

In [ ]:
block_size = 512 # I am assuming this is the size of the block that will be loaded into the training loop

print('tokenizer max len', tokenizer.max_len, 'tokenizer max len single sentence')

block_size = block_size - (tokenizer.max_len - tokenizer.max_len_single_sentence)

directory = "/content/drive/My Drive/WJ/"
filename ='film_text.txt'

block_size, directory, filename

tokenizer max len 1024 tokenizer max len single sentence


(512, '/content/drive/My Drive/WJ/', 'film_text.txt')

In [ ]:
cached_features_file = os.path.join(directory, "gpt2" + "_" + str(block_size) + "_" + filename)
cached_features_file

'/content/drive/My Drive/WJ/gpt2_512_film_text.txt'

In [ ]:
overwrite_cache = False
logger = logging.getLogger(__name__)


if os.path.exists(cached_features_file) and not overwrite_cache: #if it already exists, don't overwrite if overwite_cache set to False
      print('Loading featues from cached file')
      logger.info(f"Loading features from your cached file {cached_features_file}") # report event

      with open(cached_features_file, "rb") as cache:
                self.examples = pickle.load(cache) #take binary data and deserialize to use 
                logger.debug("Loaded examples from cache")

else:
      logger.info(f"Creating features from file {filename} at {directory}") #report event
      print('Creating features from file')

     

Creating features from file


Let's  breakdown what is happening inside the second else:

In [ ]:
self.examples = []

with open(FILE_PATH, encoding="utf-8") as f:
    text = f.read()
    print('read_file')
    logger.debug("Succesfully read text from file")

#tokenize the text
# convert_tokens_to_ids = Converts a token string (or a sequence of tokens) in a single integer id (or sequence of ids), using the vocabulary
tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

read_file


In [ ]:
len(tokenized_text)

28066436

In [ ]:
examples = []

for i in range(0, len(tokenized_text)-block_size +1, block_size):
      examples.append(
      tokenizer.build_inputs_with_special_tokens(
          tokenized_text[i:i+block_size]
      )
  )


In [ ]:
len(examples)

54817

In [ ]:
examples[9]

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

device

'cuda'